## EfficientNetB0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten, Dropout, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = tf.keras.applications.EfficientNetB0(
    weights='imagenet',
    include_top = False,
    input_shape=(224,224,3),
    # drop_connect_rate=0.4
)

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
conv_base.trainable = False

In [ ]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block7a_expand_conv':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

for layer in conv_base.layers:
  print(layer.name,layer.trainable)

input_1 False
rescaling False
normalization False
tf.math.truediv False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_dwconv False
block2b_bn False
block2b_activation False
block2b_se_squeeze False
block2b_se_reshape False
block2b_se_reduce False
block2b_se_expand False
block2b_se_excite False
block2b_

In [ ]:
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
model = Sequential()

model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 6)                 7686      
                                                                 
Total params: 4,062,377
Trainable params: 10,246
Non-tra

In [ ]:
train_dir = '/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/train'
# test_dir = '/content/drive/MyDrive/Datasets/2_intel/images/train_val_test/seg_test'
val_dir = '/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/val'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = train_dir,
    labels='inferred',
    label_mode = 'categorical',
    batch_size=32,
    image_size=(224,224)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = val_dir,
    labels='inferred',
    label_mode = 'categorical',
    batch_size=32,
    image_size=(224,224)
)

# test_ds = keras.utils.image_dataset_from_directory(
#     directory = test_dir,
#     labels='inferred',
#     label_mode = 'int',
#     batch_size=32,
#     image_size=(260,260)
# )



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
# test_ds = test_ds.map(process)

Found 2398 files belonging to 6 classes.
Found 602 files belonging to 6 classes.


In [ ]:
# model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# "categorical_crossentropy"
model.compile(optimizer='adam',loss = "categorical_crossentropy",metrics=['accuracy'])


from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(train_ds,epochs=40,validation_data=validation_ds,callbacks=callbacks_list)


Epoch 1/40
75/75 [==============================] - ETA: 0s - loss: 1.8966 - accuracy: 0.1918 
Epoch 1: val_accuracy improved from -inf to 0.18439, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-01-0.18.h5
75/75 [==============================] - 1253s 15s/step - loss: 1.8966 - accuracy: 0.1918 - val_loss: 1.8002 - val_accuracy: 0.1844
Epoch 2/40
75/75 [==============================] - ETA: 0s - loss: 1.8297 - accuracy: 0.2148
Epoch 2: val_accuracy did not improve from 0.18439
75/75 [==============================] - 8s 107ms/step - loss: 1.8297 - accuracy: 0.2148 - val_loss: 1.7832 - val_accuracy: 0.1761
Epoch 3/40
75/75 [==============================] - ETA: 0s - loss: 1.8079 - accuracy: 0.2285
Epoch 3: val_accuracy improved from 0.18439 to 0.18605, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-03-0.19.h5
75/75 [==============================] - 9s 112ms/step - los

## convnextsmall

In [ ]:
conv_base = tf.keras.applications.ConvNeXtSmall(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3),
    # drop_connect_rate=0.4
)

AttributeError: ignored

In [ ]:
conv_base = tf.keras.applications.convnext.ConvNeXtSmall(
    model_name='convnext_small',
    include_top=False,
    include_preprocessing=True,
    weights='imagenet'
)

AttributeError: ignored

## RegNetY320

In [ ]:
conv_base = tf.keras.applications.regnet.RegNetY320(
    model_name='regnety320',
    include_top=False,
    input_shape=(224, 224, 3),
    include_preprocessing=True,
    weights='imagenet'
)

In [ ]:
conv_base.summary()

Model: "regnety320"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 regnety320_prestem_rescaling (  (None, 224, 224, 3)  0          ['input_5[0][0]']                
 Rescaling)                                                                                       
                                                                                                  
 regnety320_stem_conv (Conv2D)  (None, 112, 112, 32  864         ['regnety320_prestem_rescaling[0]
                                )                                [0]']                   

In [ ]:
conv_base.trainable = False

In [ ]:
# conv_base.trainable = True

# set_trainable = False

# for layer in conv_base.layers:
#   if layer.name == 'block7a_expand_conv':
#     set_trainable = True
#   if set_trainable:
#     layer.trainable = True
#   else:
#     layer.trainable = False

# for layer in conv_base.layers:
#   print(layer.name,layer.trainable)

In [ ]:
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
model = Sequential()

model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))
# model.add(Dense(2048,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

In [ ]:
train_dir = '/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/train'
test_dir = '/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize'
val_dir = '/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/val'

In [ ]:
# generators
IMG_SIZE = 150
label_mode = 'int'
train_ds = keras.utils.image_dataset_from_directory(
    directory = train_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = val_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)



Found 2398 files belonging to 6 classes.
Found 602 files belonging to 6 classes.
Found 1439 files belonging to 6 classes.


In [ ]:
# model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# "categorical_crossentropy"
model.compile(optimizer='adam',loss = "categorical_crossentropy",metrics=['accuracy'])


from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(train_ds,epochs=40,validation_data=validation_ds,callbacks=callbacks_list)


Epoch 1/40
75/75 [==============================] - ETA: 0s - loss: 1.4741 - accuracy: 0.4771
Epoch 1: val_accuracy improved from -inf to 0.17276, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-01-0.17.h5
75/75 [==============================] - 64s 749ms/step - loss: 1.4741 - accuracy: 0.4771 - val_loss: 1.8147 - val_accuracy: 0.1728
Epoch 2/40
75/75 [==============================] - ETA: 0s - loss: 1.1916 - accuracy: 0.5730
Epoch 2: val_accuracy did not improve from 0.17276
75/75 [==============================] - 52s 686ms/step - loss: 1.1916 - accuracy: 0.5730 - val_loss: 1.9911 - val_accuracy: 0.1678
Epoch 3/40
75/75 [==============================] - ETA: 0s - loss: 1.0816 - accuracy: 0.5913
Epoch 3: val_accuracy improved from 0.17276 to 0.17608, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/ENB0_weights-improvement-03-0.18.h5
75/75 [==============================] - 55s 735ms/step - lo

In [ ]:
history = model.fit(train_ds,epochs=40,validation_data=validation_ds,callbacks=callbacks_list)

Epoch 1/40
75/75 [==============================] - ETA: 0s - loss: 0.3038 - accuracy: 0.8903
Epoch 1: val_accuracy did not improve from 0.68106
75/75 [==============================] - 51s 671ms/step - loss: 0.3038 - accuracy: 0.8903 - val_loss: 1.2066 - val_accuracy: 0.6329
Epoch 2/40
75/75 [==============================] - ETA: 0s - loss: 0.2990 - accuracy: 0.8837
Epoch 2: val_accuracy did not improve from 0.68106
75/75 [==============================] - 53s 699ms/step - loss: 0.2990 - accuracy: 0.8837 - val_loss: 1.2441 - val_accuracy: 0.6462
Epoch 3/40
75/75 [==============================] - ETA: 0s - loss: 0.2789 - accuracy: 0.9008
Epoch 3: val_accuracy did not improve from 0.68106
75/75 [==============================] - 53s 708ms/step - loss: 0.2789 - accuracy: 0.9008 - val_loss: 1.3100 - val_accuracy: 0.6362
Epoch 4/40
75/75 [==============================] - ETA: 0s - loss: 0.3091 - accuracy: 0.8903
Epoch 4: val_accuracy did not improve from 0.68106
75/75 [=================

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/final_weights-improvement-03-0.88.hdf5')

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              8389632   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                      

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_ds,batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


45/45 [==============================] - 120s 2s/step - loss: 0.9476 - accuracy: 0.7290
test loss, test acc: [0.9475961327552795, 0.7289784550666809]


In [ ]:
import glob

In [ ]:
len(glob.glob('/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize/*/*'))

1848

## Preprocessing

### imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten, Dropout, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small

import glob

### Paths

In [ ]:
train_dir = "/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/train"
val_dir = "/content/drive/MyDrive/Datasets/2_intel/images/from_ori_test_create_train_val/val"
test_dir = "/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize"

### Preprocessing image_dataset_from_directory

In [ ]:
label_mode = 'int'
IMG_SIZE = 224
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = train_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = val_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=32,
    image_size=(IMG_SIZE,IMG_SIZE)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)



Found 2398 files belonging to 6 classes.
Found 602 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


## tf.keras.applications.mobilenet_v3

In [ ]:
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small

In [ ]:
conv_base = MobileNetV3Small(input_shape=(IMG_SIZE,IMG_SIZE,3), classes=6,weights="imagenet", pooling=None, include_top=False)

In [ ]:
conv_base.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 150, 150, 3)  0           ['input_4[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 75, 75, 16)   432         ['rescaling_3[0][0]']            
                                                                                                  
 Conv/BatchNorm (BatchNormaliza  (None, 75, 75, 16)  64          ['Conv[0][0]']    

In [ ]:
conv_base = MobileNetV3Small(weights="imagenet", pooling=None, include_top=True)

10734624/10734624 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_2 (Rescaling)        (None, None, None,   0           ['input_3[0][0]']                
                                3)                                                                
                                                                                                  
 Conv (Conv2D)                  (None, None, None,   432         ['rescaling_2[0][0]']            
                                16)                                                

In [ ]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'expanded_conv_10/expand':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

for layer in conv_base.layers:
  print(layer.name,layer.trainable)

input_4 False
rescaling_3 False
Conv False
Conv/BatchNorm False
tf.__operators__.add_82 False
re_lu_97 False
tf.math.multiply_82 False
multiply_55 False
expanded_conv/depthwise/pad False
expanded_conv/depthwise False
expanded_conv/depthwise/BatchNorm False
re_lu_98 False
expanded_conv/squeeze_excite/AvgPool False
expanded_conv/squeeze_excite/Conv False
expanded_conv/squeeze_excite/Relu False
expanded_conv/squeeze_excite/Conv_1 False
tf.__operators__.add_83 False
re_lu_99 False
tf.math.multiply_83 False
expanded_conv/squeeze_excite/Mul False
expanded_conv/project False
expanded_conv/project/BatchNorm False
expanded_conv_1/expand False
expanded_conv_1/expand/BatchNorm False
re_lu_100 False
expanded_conv_1/depthwise/pad False
expanded_conv_1/depthwise False
expanded_conv_1/depthwise/BatchNorm False
re_lu_101 False
expanded_conv_1/project False
expanded_conv_1/project/BatchNorm False
expanded_conv_2/expand False
expanded_conv_2/expand/BatchNorm False
re_lu_102 False
expanded_conv_2/depthwi

In [ ]:
model = Sequential()

model.add(conv_base)
# model.add(GlobalAveragePooling2D())
# model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
# model.add(Dense(2048,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(1024,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(6,activation='softmax'))

In [ ]:
# model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# "categorical_crossentropy"
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])


from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/MNV3_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(train_ds,epochs=40,validation_data=validation_ds,callbacks=callbacks_list)


Epoch 1/40
73/75 [============================>.] - ETA: 0s - loss: 1.6709 - accuracy: 0.3733
Epoch 1: val_accuracy improved from -inf to 0.23920, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/MNV3_weights-improvement-01-0.24.h5
75/75 [==============================] - 10s 88ms/step - loss: 1.6667 - accuracy: 0.3753 - val_loss: 2.1838 - val_accuracy: 0.2392
Epoch 2/40
73/75 [============================>.] - ETA: 0s - loss: 1.2924 - accuracy: 0.5073
Epoch 2: val_accuracy improved from 0.23920 to 0.28073, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/ENB0/MNV3_weights-improvement-02-0.28.h5
75/75 [==============================] - 7s 90ms/step - loss: 1.2836 - accuracy: 0.5100 - val_loss: 3.0258 - val_accuracy: 0.2807
Epoch 3/40
73/75 [============================>.] - ETA: 0s - loss: 1.1772 - accuracy: 0.5479
Epoch 3: val_accuracy did not improve from 0.28073
75/75 [==============================] - 6s 75ms/step - loss: 1

KeyboardInterrupt: ignored

## Evaluate Model using Test Data

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-06-0.90.h5')

In [ ]:
test_dir = '/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize'

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = 'int',
    batch_size=64,
    image_size=(331,331)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

# train_ds = train_ds.map(process)
# validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)

Found 3000 files belonging to 6 classes.


In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_ds,batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


47/47 [==============================] - 1187s 20s/step - loss: 0.8087 - sparse_categorical_accuracy: 0.7687
test loss, test acc: [0.80865877866745, 0.768666684627533]


## NNL Evaluate

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-27-0.91.h5')


In [ ]:
test_dir = '/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize'

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = 'int',
    batch_size=64,
    image_size=(331,331)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

# train_ds = train_ds.map(process)
# validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)

Found 3000 files belonging to 6 classes.


In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_ds,batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


47/47 [==============================] - 1243s 21s/step - loss: 0.8248 - sparse_categorical_accuracy: 0.7773
test loss, test acc: [0.8247968554496765, 0.7773333191871643]


In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-27-0.91.h5')model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-27-0.91.h5')  dfsgdddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd